# Computational scheme for the free energy of a simplicial complex
*Cyril Rommens, s12495719, masterproject MSc Physics and Astronomy: Computational Physics of Complex Systems*

**Introduction**
In this notebook, we compute the Free energy of a simplicial complex from a given dataset. First we analyse the dataset to define a connection probability. After this, we can choose a desired simplicial complex G_i from the set with $0<i<N$ complexes and compute the internal energy U and the entropy S, according to Knill's work. From this we can then compute the Helmholtz free energy F.

**Import libraries**

In [13]:
# Basic data manipulation libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import timeit
import os

**Import the dataset**

In [14]:
# For now just give a pre-set dataset
import openpyxl
import numpy as np

# Specify the directory and file name
excel_file_path = r'C:\Users\cyril\OneDrive\Documenten\MSc Physics and Astronomy\Thesis\Planning\Week 9 - 18 jan\Dataset_example\SimplicialComplex_G.xlsx'

# Load the Excel file into a Pandas DataFrame
G_list = []
sheet_list = ['G_A', 'G_B', 'G_C', 'G_D', 'G_E', 'G_F']
for sheet in sheet_list:
    df = pd.read_excel(excel_file_path, sheet_name=sheet, engine='openpyxl', header=None)
    G_i = df.values.astype(np.float64)
    G_i = np.nan_to_num(G_i, nan=0)
    G_list.append(G_i)

# Now, G is a 2D NumPy array containing the data from the first sheet of the Excel file
G = np.array(G_list)

**Binarization**

Knill reports the connection matrix as binary, so L_xy = 1 if the simplex x intersects with the simplex y and L_xy = 0 if it doesn’t. So there is no degree in connectivity. Binarize the matrices in further calculations for now, to first work out the method according to Knill, without connectivity degree.

In [15]:
threshold = 0.0
G_binary = (G > threshold).astype(int)

**Compute the connection probability matrix $L_p$**

In [16]:
L_p = np.mean(G_binary, axis=0)

**Compute the inverse connection matrix ${L_i}^{-1}$**

To do this we first have to get rid of the zero rows and column, since the matrix is not invertible if the determinant of the desired matrix is zero. When we did this we can generate the inverted matrix. So for example for the first matrix L_A this would go like:

In [33]:
def inverse_matrix_generator(matrix):

    # Find non-zero rows and columns
    non_zero_rows = ~np.all(matrix == 0, axis=1)

    # Store the indices of removed rows. This is the same for the columns since the matrices are symmetric
    removed_rows = np.where(~non_zero_rows)[0].tolist()

    # Extract the non-zero rows and columns
    result_matrix = matrix[non_zero_rows][:, non_zero_rows]

    # Generate inverse matrix
    if np.linalg.det(result_matrix) != 0:
        inverse_matrix_unrounded = np.linalg.inv(result_matrix)
        inverse_matrix = np.round(inverse_matrix_unrounded, decimals=2)
    else:
        inverse_matrix = 0

    return result_matrix, removed_rows, inverse_matrix

In [34]:
# Customize print settings to print the entire matrix
#np.set_printoptions(threshold=np.inf)
# Reset NumPy print options to the default settings
#np.set_printoptions(threshold=1000)

In [63]:
# Generate desired matrix
matrix = G[0]

print("Original Matrix:")
print(matrix)

Original Matrix:
[[1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0.]
 [1. 1. 1. 1. 1. 1. 2. 1. 0. 0. 0. 1. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1. 2. 0. 0. 0. 1. 0. 0.]
 [0. 1. 1. 1. 1. 2. 1. 1. 0. 0. 0. 1. 0. 0.]
 [1. 1. 1. 1. 3. 1. 2. 1. 0. 0. 0. 1. 0. 0.]
 [1. 1. 1. 2. 1. 3. 1. 1. 0. 0. 0. 3. 0. 0.]
 [0. 2. 1. 1. 2. 1. 3. 1. 0. 0. 0. 3. 0. 0.]
 [1. 1. 2. 1. 1. 1. 1. 3. 0. 0. 0. 3. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 3. 3. 3. 0. 0. 0. 6. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
14


In [37]:
# Generate matrix without zero rows and columns
result_matrix, removed_rows, inverse_matrix = inverse_matrix_generator(matrix)

print("\nMatrix without Zero Rows and Columns:")
print(result_matrix)

print("\nIndices of Removed Rows/Columns:", removed_rows)


Matrix without Zero Rows and Columns:
[[1. 1. 0. 0. 1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1. 1. 2. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 2. 1.]
 [0. 1. 1. 1. 1. 2. 1. 1. 1.]
 [1. 1. 1. 1. 3. 1. 2. 1. 1.]
 [1. 1. 1. 2. 1. 3. 1. 1. 3.]
 [0. 2. 1. 1. 2. 1. 3. 1. 3.]
 [1. 1. 2. 1. 1. 1. 1. 3. 3.]
 [1. 1. 1. 1. 1. 3. 3. 3. 6.]]

Indices of Removed Rows/Columns: [8, 9, 10, 12, 13]


In [38]:
# Generate the inverse matrix
if np.linalg.det(result_matrix) != 0:
    print("Inverse Matrix:")
    print(inverse_matrix)
else:
    print("The matrix is singular and cannot be inverted.")

Inverse Matrix:
[[ 0.29  0.56 -0.31 -0.31  0.    0.1  -0.19  0.1  -0.04]
 [ 0.56  0.21  0.04  0.04 -0.5   0.02  0.46  0.02 -0.31]
 [-0.31  0.04 -1.04  0.96  0.   -0.27  0.04  0.73 -0.19]
 [-0.31  0.04  0.96 -1.04  0.    0.73  0.04 -0.27 -0.19]
 [-0.   -0.5   0.    0.    0.5  -0.   -0.   -0.   -0.  ]
 [ 0.1   0.02 -0.27  0.73  0.   -0.13 -0.23 -0.13  0.15]
 [-0.19  0.46  0.04  0.04 -0.   -0.23 -0.04 -0.23  0.19]
 [ 0.1   0.02  0.73 -0.27  0.   -0.13 -0.23 -0.13  0.15]
 [-0.04 -0.31 -0.19 -0.19 -0.    0.15  0.19  0.15  0.04]]


Repeat this for the complete dataset, so that we end up with inverse matrices for $G_A$ to $G_F$

In [39]:
L_inverse_list = []
removed_rows_list = []

for matrix in G:
    result_matrix, removed_rows, inverse_matrix = inverse_matrix_generator(matrix)
    L_inverse_list.append(inverse_matrix)
    removed_rows_list.append(removed_rows)

Try to decompress the inverse matrices for later use, since we need to multiple them with the $L_p$ matrix and thus should be of equal dimensions.
However, decompressing is not that easy, because the stored indexation of the removed rows and columns is not the same as the place we should return zero rows and columns. To do this, below, we first decompressed the first row of the ${L_i}^{-1}$ matrix by inserting zeros at the indices of the removed columns. This decompressed row list then presents the location where zero columns (and rows due to symmetry) should be inserted into the ${L_i}^{-1}$ matrix. This happens in the next step, using the function add_zero_array for each zero value in the decompressed row list.

In [73]:
# Decompress the compressed array with inserting zeros at the indices of the removed rows and columns
def decompress(array_length, index_removed):
    
    # Add +1 to each value in the index_list since the count starts at zero
    index_removed = [x + 1 for x in index_removed]
    
    array_compressed = [item for item in range(0, array_length) if item not in index_removed]
    zero_list = [0]*array_length

    for index in array_compressed:
        zero_list[index-1]=index
    
    array_decompressed = zero_list

    return array_decompressed

In [71]:
# Add a zero column at the desired index
def add_zero_array(original_array, index, axis_choice):
    zero_array = np.zeros(original_array.shape[0], dtype=original_array.dtype)
    inserted_array = np.insert(original_array, index, zero_array, axis=axis_choice)
    return inserted_array

In [81]:
complex = L_inverse_list[0]

# axis choice definition
column_axis = 0
row_axis = 1

# Decompress the 1D array
original_array_length = 14
decompressed_1D_array = decompress(original_array_length, removed_rows_list[0])

# Insert zero rows/columns at the indices with value zero from the decompressed 1D array
for index in range(0, len(decompressed_1D_array)):
    if decompressed_1D_array[index] == 0:
        inserted_column_array = add_zero_array(complex, index, column_axis)
        inserted_row_array = add_zero_array(inserted_column_array, index, row_axis)
        complex = inserted_row_array

print("Complete decompressed 2D array:")
print(complex)

Complete decompressed 2D array:
[[ 0.29  0.56 -0.31 -0.31  0.    0.1  -0.19  0.1   0.    0.    0.   -0.04
   0.    0.  ]
 [ 0.56  0.21  0.04  0.04 -0.5   0.02  0.46  0.02  0.    0.    0.   -0.31
   0.    0.  ]
 [-0.31  0.04 -1.04  0.96  0.   -0.27  0.04  0.73  0.    0.    0.   -0.19
   0.    0.  ]
 [-0.31  0.04  0.96 -1.04  0.    0.73  0.04 -0.27  0.    0.    0.   -0.19
   0.    0.  ]
 [-0.   -0.5   0.    0.    0.5  -0.   -0.   -0.    0.    0.    0.   -0.
   0.    0.  ]
 [ 0.1   0.02 -0.27  0.73  0.   -0.13 -0.23 -0.13  0.    0.    0.    0.15
   0.    0.  ]
 [-0.19  0.46  0.04  0.04 -0.   -0.23 -0.04 -0.23  0.    0.    0.    0.19
   0.    0.  ]
 [ 0.1   0.02  0.73 -0.27  0.   -0.13 -0.23 -0.13  0.    0.    0.    0.15
   0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0. 